# Comprehensive Guide to Method Overriding and Hiding in Java

## Table of Contents
1. [Instance Method Overriding](#instance-method-overriding)
2. [Static Method Hiding](#static-method-hiding)
3. [Interface Methods](#interface-methods)
4. [Access Modifiers Rules](#access-modifiers-rules)
5. [Summary Table](#summary-table)

---

## Instance Method Overriding

### What is Method Overriding?

**Method overriding** occurs when a subclass provides a specific implementation for a method that already exists in its superclass. The overriding method must have:
- The same name
- The same number and type of parameters (same signature)
- The same return type (or a covariant return type)

### Why Override Methods?

Overriding allows a subclass to inherit behavior from a superclass that is "close enough" and then modify it as needed for specific requirements.

### Covariant Return Types

An overriding method can return a **subtype** of the return type declared in the superclass method. This is called a covariant return type.

**Example:**
```java
class Animal {
    public Animal reproduce() {
        return new Animal();
    }
}

class Dog extends Animal {
    @Override
    public Dog reproduce() {  // Dog is a subtype of Animal
        return new Dog();
    }
}
```

### The @Override Annotation

The `@Override` annotation instructs the compiler to verify that you're actually overriding a superclass method. If the method doesn't exist in the superclass, the compiler generates an error.

**Example:**
```java
class Vehicle {
    public void start() {
        System.out.println("Vehicle starting");
    }
}

class Car extends Vehicle {
    @Override  // Recommended practice
    public void start() {
        System.out.println("Car starting");
    }
}
```

### Key Behavior

**The version of the overridden instance method that gets invoked is always the one in the subclass**, regardless of the reference type used.

---

## Static Method Hiding

### What is Method Hiding?

When a subclass defines a **static method** with the same signature as a static method in the superclass, the subclass method **hides** (not overrides) the superclass method.

### Critical Difference from Overriding

**The version of the hidden static method that gets invoked depends on whether it is invoked from the superclass or the subclass** (determined at compile-time, not runtime).

### Complete Example

```java
public class Animal {
    public static void testClassMethod() {
        System.out.println("The static method in Animal");
    }
    
    public void testInstanceMethod() {
        System.out.println("The instance method in Animal");
    }
}

public class Cat extends Animal {
    public static void testClassMethod() {
        System.out.println("The static method in Cat");
    }
    
    @Override
    public void testInstanceMethod() {
        System.out.println("The instance method in Cat");
    }

    public static void main(String[] args) {
        Cat myCat = new Cat();
        Animal myAnimal = myCat;  // Cat object with Animal reference
        Animal.testClassMethod();      // Calls Animal's static method
        myAnimal.testInstanceMethod(); // Calls Cat's instance method
    }
}
```

**Output:**
```
The static method in Animal
The instance method in Cat
```

### Why This Happens

- **Static methods** are resolved at **compile-time** based on the reference type
- **Instance methods** are resolved at **runtime** based on the actual object type (polymorphism)

---

## Interface Methods

### Default and Abstract Methods

Default methods and abstract methods in interfaces are inherited like instance methods. However, conflicts can arise when multiple interfaces provide methods with the same signature.

### Inheritance Resolution Rules

Java follows two key principles to resolve naming conflicts:

#### **Principle 1: Instance Methods Are Preferred Over Interface Default Methods**

**Example:**
```java
public class Horse {
    public String identifyMyself() {
        return "I am a horse.";
    }
}

public interface Flyer {
    default public String identifyMyself() {
        return "I am able to fly.";
    }
}

public interface Mythical {
    default public String identifyMyself() {
        return "I am a mythical creature.";
    }
}

public class Pegasus extends Horse implements Flyer, Mythical {
    public static void main(String... args) {
        Pegasus myApp = new Pegasus();
        System.out.println(myApp.identifyMyself());
    }
}
```

**Output:** `I am a horse.`

The inherited instance method from `Horse` takes precedence over the default methods in `Flyer` and `Mythical`.

---

#### **Principle 2: Already Overridden Methods Are Ignored**

When supertypes share a common ancestor, methods already overridden by other candidates are ignored.

**Example:**
```java
public interface Animal {
    default public String identifyMyself() {
        return "I am an animal.";
    }
}

public interface EggLayer extends Animal {
    @Override
    default public String identifyMyself() {
        return "I am able to lay eggs.";
    }
}

public interface FireBreather extends Animal { }

public class Dragon implements EggLayer, FireBreather {
    public static void main(String... args) {
        Dragon myApp = new Dragon();
        System.out.println(myApp.identifyMyself());
    }
}
```

**Output:** `I am able to lay eggs.`

`EggLayer` overrides `Animal.identifyMyself()`, so `Dragon` inherits from `EggLayer`. `FireBreather` doesn't override it, so it's ignored.

---

### Handling Conflicts

If two or more independently defined default methods conflict, you **must explicitly override** the method.

**Example:**
```java
public interface OperateCar {
    default public int startEngine(EncryptedKey key) {
        // Implementation
        return 0;
    }
}

public interface FlyCar {
    default public int startEngine(EncryptedKey key) {
        // Implementation
        return 1;
    }
}

public class FlyingCar implements OperateCar, FlyCar {
    @Override
    public int startEngine(EncryptedKey key) {
        // You can call specific interface implementations
        FlyCar.super.startEngine(key);
        OperateCar.super.startEngine(key);
        return 2;
    }
}
```

The syntax `InterfaceName.super.methodName()` allows you to invoke a specific interface's default implementation.

---

### Instance Methods Override Abstract Interface Methods

Inherited instance methods from classes can override abstract interface methods.

**Example:**
```java
public interface Mammal {
    String identifyMyself();  // Abstract method
}

public class Horse {
    public String identifyMyself() {
        return "I am a horse.";
    }
}

public class Mustang extends Horse implements Mammal {
    public static void main(String... args) {
        Mustang myApp = new Mustang();
        System.out.println(myApp.identifyMyself());
    }
}
```

**Output:** `I am a horse.`

The `Horse` class's instance method satisfies the `Mammal` interface requirement.

---

### Important Note

**Static methods in interfaces are never inherited.**

---

## Access Modifiers Rules

### Overriding and Access Levels

The access specifier for an overriding method can allow **more access, but not less**, than the overridden method.

**Valid Examples:**
```java
class Parent {
    protected void display() {
        System.out.println("Parent");
    }
}

class Child extends Parent {
    @Override
    public void display() {  // More accessible (protected → public) ✓
        System.out.println("Child");
    }
}
```

**Invalid Example:**
```java
class Parent {
    public void display() {
        System.out.println("Parent");
    }
}

class Child extends Parent {
    @Override
    protected void display() {  // Less accessible (public → protected) ✗
        System.out.println("Child");
    }
}
```

### Access Level Hierarchy

```
public > protected > package-private (default) > private
```

### Mixing Instance and Static Methods

**You cannot:**
- Change an instance method to a static method
- Change a static method to an instance method

Both will result in **compile-time errors**.

---

## Summary Table

### Defining a Method with the Same Signature as a Superclass's Method

|                          | **Superclass Instance Method** | **Superclass Static Method** |
|--------------------------|-------------------------------|------------------------------|
| **Subclass Instance Method** | **Overrides**                 | **Compile-time Error**       |
| **Subclass Static Method**   | **Compile-time Error**        | **Hides**                    |

---

## Method Overloading vs. Overriding

### Important Distinction

In a subclass, you can **overload** methods inherited from the superclass by changing the signature (different parameters). These overloaded methods:
- **Do NOT hide** the superclass methods
- **Do NOT override** the superclass methods
- Are **new methods, unique to the subclass**

**Example:**
```java
class Calculator {
    public int add(int a, int b) {
        return a + b;
    }
}

class ScientificCalculator extends Calculator {
    // Overloading (different signature)
    public double add(double a, double b) {
        return a + b;
    }
    
    // This would be overriding (same signature)
    @Override
    public int add(int a, int b) {
        return a + b + 1;
    }
}
```

---

## Key Takeaways

1. **Instance methods** are overridden and resolved at **runtime** (polymorphism)
2. **Static methods** are hidden and resolved at **compile-time** (no polymorphism)
3. **Interface default methods** follow specific inheritance rules with clear precedence
4. **Access modifiers** can only become more permissive when overriding
5. **Never mix static and instance** methods with the same signature
6. Use **@Override** annotation to catch errors at compile-time
7. **Overloading** creates new methods; it doesn't override or hide